In [21]:
import pandas as pd
import numpy as np
import os

from datetime import datetime
import torch
from chronos import Chronos2Pipeline

In [23]:
pipeline = Chronos2Pipeline.from_pretrained("amazon/chronos-2",  device_map="cuda")

In [3]:
df_survontie = pd.read_csv('cleaned_datasets/LHT/Survontie.csv')

In [4]:
df_survontie["Timestamp"] = pd.to_datetime(df_survontie["Timestamp"],format="%Y-%m-%d %H:%M:%S", errors="coerce")
print("NaT after parse:", df_survontie["Timestamp"].isna().sum())
# Sort the ds by timestamp
df_survontie = df_survontie.sort_values(by="Timestamp").reset_index(drop=True)
df_survontie.head()

NaT after parse: 0


,Timestamp,Temperature_C,Humidity
0,2021-01-20 00:00:00,-2.09,100.0
1,2021-01-20 01:00:00,-2.09,100.0
2,2021-01-20 02:00:00,-2.07,100.0
3,2021-01-20 03:00:00,-1.92,100.0
4,2021-01-20 04:00:00,-1.84,100.0


In [36]:
df_survontie["id"] = "survontie"

context_data = df_survontie[["id", "Timestamp", 'Temperature_C', 'Humidity']]

In [33]:
target = ["Temperature_C", "Humidity"]
prediction_survontie = pipeline.predict_df(
    context_data,              
    future_df=None,
    prediction_length=1024,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column="id",
    timestamp_column="Timestamp",
    target=target,
)

In [34]:
wide_pred = (
    prediction_survontie
    .pivot(index=["Timestamp", "id"], columns="target_name", values="predictions")
    .reset_index()
)

wide_pred.tail()


target_name,Timestamp,id,Humidity,Temperature_C
1019,2025-10-30 11:00:00,survontie,99.684380,13.341316
1020,2025-10-30 12:00:00,survontie,99.616135,13.646994
1021,2025-10-30 13:00:00,survontie,99.677017,14.036665
1022,2025-10-30 14:00:00,survontie,99.555466,14.417502
1023,2025-10-30 15:00:00,survontie,99.633408,14.424516


In [28]:
df_Keltimaentie= pd.read_csv('cleaned_datasets\LHT\Keltimaentie.csv')

<>:1: SyntaxWarning: invalid escape sequence '\L'
<>:1: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Home\AppData\Local\Temp\ipykernel_544\3840567988.py:1: SyntaxWarning: invalid escape sequence '\L'
  df_Keltimaentie= pd.read_csv('cleaned_datasets\LHT\Keltimaentie.csv')


In [29]:
df_Keltimaentie["Timestamp"] = pd.to_datetime(df_Keltimaentie["Timestamp"],format="%Y-%m-%d %H:%M:%S", errors="coerce")
print("NaT after parse:", df_Keltimaentie["Timestamp"].isna().sum())
# Sort the ds by timestamp
df_Keltimaentie = df_Keltimaentie.sort_values(by="Timestamp").reset_index(drop=True)
df_Keltimaentie.head()

NaT after parse: 0


,Timestamp,Temperature_C,Humidity
0,2021-01-20 00:00:00,-2.24,100.0
1,2021-01-20 01:00:00,-2.26,100.0
2,2021-01-20 02:00:00,-2.04,100.0
3,2021-01-20 03:00:00,-1.87,100.0
4,2021-01-20 04:00:00,-1.95,100.0


In [58]:
df_Keltimaentie.describe()

,Timestamp,Temperature_C,Humidity
count,40848,40769.000000,40769.000000
mean,2023-05-20 23:30:00,5.373171,88.921404
min,2021-01-20 00:00:00,-30.910000,22.400000
25%,2022-03-21 11:45:00,-2.270000,83.550000
50%,2023-05-20 23:30:00,4.550000,100.000000
75%,2024-07-19 11:15:00,13.465114,100.000000
max,2025-09-17 23:00:00,36.640000,100.000000
std,NaN,11.037242,18.016773


In [ ]:
df_Keltimaentie["id"] = "Keltimaentie"

df_Keltimaentie_data = df_Keltimaentie[["id", "Timestamp", 'Temperature_C', 'Humidity']]

In [65]:
targets = ["Temperature_C", "Humidity"]
prediction_Keltimaentie = pipeline.predict_df(
    df_Keltimaentie_data,              
    future_df=None,
    prediction_length=1700,
    id_column="id",
    timestamp_column="Timestamp",
    target=targets,
)

d:\Fiveth\Marjetas_project\.venv312\Lib\site-packages\chronos\chronos2\pipeline.py:514: UserWarning: We recommend keeping prediction length <= 1024. The quality of longer predictions may degrade since the model is not optimized for it. 
  warnings.warn(msg)


In [66]:
wide_pred = (prediction_Keltimaentie
    .pivot(index=["Timestamp", "id"], columns="target_name", values="predictions")
    .reset_index())

wide_pred.tail()

target_name,Timestamp,id,Humidity,Temperature_C
1695,2025-11-27 15:00:00,Keltimaentie,99.763779,14.076080
1696,2025-11-27 16:00:00,Keltimaentie,99.750366,13.685637
1697,2025-11-27 17:00:00,Keltimaentie,99.739410,13.382821
1698,2025-11-27 18:00:00,Keltimaentie,99.755081,12.959631
1699,2025-11-27 19:00:00,Keltimaentie,99.766075,12.347520


In [67]:
wind_df = pd.read_csv('cleaned_datasets/cleaned_wind_data.csv')

In [69]:
wind_df["Timestamp"] = pd.to_datetime(wind_df["Timestamp"],format="%Y-%m-%d %H:%M:%S", errors="coerce")
print("NaT after parse:", wind_df["Timestamp"].isna().sum())
# Sort the ds by timestamp
wind_df = wind_df.sort_values(by="Timestamp").reset_index(drop=True)
wind_df.head()

NaT after parse: 0


,snow_depth (m),wind_speed_10m (km/h),wind_direction_10m (°),wind_gusts_10m (km/h),surface_pressure (hPa),Timestamp
0,0.22,1.3,304,2.9,990.9,2021-01-01 00:00:00
1,0.22,2.5,225,5.0,991.2,2021-01-01 01:00:00
2,0.22,4.8,222,9.7,991.2,2021-01-01 02:00:00
3,0.22,6.1,230,11.9,991.2,2021-01-01 03:00:00
4,0.22,7.9,227,16.2,991.5,2021-01-01 04:00:00


In [72]:
wind_df.columns

Index(['snow_depth (m)', 'wind_speed_10m (km/h)', 'wind_direction_10m (°)',
       'wind_gusts_10m (km/h)', 'surface_pressure (hPa)', 'Timestamp', 'id'],
      dtype='object')

In [73]:
wind_df["id"] = "wind"

wind_data = wind_df[["id", "Timestamp", 'snow_depth (m)', 'wind_speed_10m (km/h)', 'wind_direction_10m (°)',
       'wind_gusts_10m (km/h)', 'surface_pressure (hPa)']]

In [74]:
target = ['snow_depth (m)', 'wind_speed_10m (km/h)', 'wind_direction_10m (°)',
       'wind_gusts_10m (km/h)', 'surface_pressure (hPa)']

In [75]:

prediction_wind= pipeline.predict_df(
    wind_data,              
    future_df=None,
    prediction_length=1700,
    id_column="id",
    timestamp_column="Timestamp",
    target=target,
)

d:\Fiveth\Marjetas_project\.venv312\Lib\site-packages\chronos\chronos2\pipeline.py:514: UserWarning: We recommend keeping prediction length <= 1024. The quality of longer predictions may degrade since the model is not optimized for it. 
  warnings.warn(msg)


In [76]:
wide_pred = (prediction_wind
    .pivot(index=["Timestamp", "id"], columns="target_name", values="predictions")
    .reset_index())

wide_pred.tail()

target_name,Timestamp,id,snow_depth (m),surface_pressure (hPa),wind_direction_10m (°),wind_gusts_10m (km/h),wind_speed_10m (km/h)
1695,2025-11-28 15:00:00,wind,0.001838,996.137634,186.745041,27.221378,11.556326
1696,2025-11-28 16:00:00,wind,0.002301,995.878845,187.850342,24.796442,10.728363
1697,2025-11-28 17:00:00,wind,0.001912,995.948303,187.721680,22.564779,10.305304
1698,2025-11-28 18:00:00,wind,0.002024,995.904541,187.357376,20.967770,10.094626
1699,2025-11-28 19:00:00,wind,0.002399,995.886414,187.603378,19.969454,9.891026
